In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
from scipy.stats import norm

In [ ]:
import sys
sys.path.insert(0, '/home/jg2447/slayman/perturb/dagma/')
from utils import *

### overlap with len2 ChIP

In [ ]:
chip = pd.read_csv("../../result/network/ChIP_TIP.tsv", header=None, sep="\t")
chip.columns = ['source', 'target']
chip = nx.from_pandas_edgelist(chip, create_using=nx.DiGraph)
chip_adj = nx.to_pandas_adjacency(chip)

A_chip = chip_adj.copy()
for ii in set(list(range(2319))) - set(chip_adj.index):
    A_chip[ii] = 0
A_chip = A_chip.T
for ii in set(list(range(2319))) - set(chip_adj.index):
    A_chip[ii] = 0
A_chip = A_chip.T
A_chip = A_chip.loc[list(range(2319)), list(range(2319))].copy()

In [ ]:
dagma = pd.read_csv("../../result/network/DAGMA_DAG.tsv", header=None, sep="\t")
count = 0
A2 = A_chip @ A_chip
for pair in dagma.values:
    if A2.loc[pair[0], pair[1]] > 0:
        count += 1
print(count)

### overlap with len2 ChIP+Biogrid

In [ ]:
biogrid = pd.read_csv("../../result/network/BIOGRID.tsv", header=None, sep="\t")
biogrid.columns = ['source', 'target']
biogrid = nx.from_pandas_edgelist(biogrid, create_using=nx.DiGraph)
biogrid_adj = nx.to_pandas_adjacency(biogrid)

A_biogrid = biogrid_adj.copy()
for ii in set(list(range(2319))) - set(biogrid_adj.index):
    A_biogrid[ii] = 0
A_biogrid = A_biogrid.T
for ii in set(list(range(2319))) - set(biogrid_adj.index):
    A_biogrid[ii] = 0
A_biogrid = A_biogrid.T
A_biogrid = A_biogrid.loc[list(range(2319)), list(range(2319))].copy()

A_combine = A_biogrid + A_chip

In [ ]:
dagma = pd.read_csv("../../result/network/DAGMA_DAG.tsv", header=None, sep="\t")
count = 0
A2 = A_combine @ A_combine
for pair in dagma.values:
    if A2.loc[pair[0], pair[1]] > 0:
        count += 1
print(count)

### overlap with len2 ChIP+random

In [ ]:
biogrid = pd.read_csv("../../result/network/BIOGRID.tsv", header=None, sep="\t")
biogrid_len = len(biogrid)
count_list = []
for ii in range(50):
    random = simulate_dag(2319, biogrid_len, "ER")
    A_combine2 = A_chip + random

    dagma = pd.read_csv("../../result/network/DAGMA_DAG.tsv", header=None, sep="\t")
    count = 0
    A2 = A_combine2 @ A_combine2
    for pair in dagma.values:
        if A2.loc[pair[0], pair[1]] > 0:
            count += 1
    count_list.append(count)

In [ ]:
Z = (126 - np.mean(count_list)) / np.std(count_list)
p_values = norm.sf(Z)

In [ ]:
p_values